In [3]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes
!pip install nltk==3.7
!pip install -U Flask
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import hashlib

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# client = storage.Client()
# bucket_name = 'ir_assg3_eithan'

# bucket = client.bucket(bucket_name)

# # Get a list of all the blobs in the folder
# blobs = bucket.list_blobs(prefix='indexes/')

# # Iterate over the list of blobs and download each one to a local file
# for blob in blobs:
#     if 'doc_norm_dict.pickle' in blob.name or 'doc_len_dict.pickle' in blob.name:
#         local_file_name = blob.name[blob.name.rfind("/")+1:]
#         blob.download_to_filename(local_file_name)

In [29]:
from flask import Flask, request, jsonify
import collections
import csv
import gzip
import math

!pip install fasttext
import fasttext
import requests

!pip install gensim
from gensim.models.wrappers import * 

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 96 from C header, got 80 from PyObject

In [6]:
%cd /home/dataproc
# from inverted_index_gcp import InvertedIndex
# from inverted_index_gcp import *
# import inverted_index_gcp as iig
from inverted_index_gcp import *

/home/dataproc


In [7]:
# import importlib
# importlib.reload(iig)

In [8]:
path_anchor = "/home/dataproc/postings_gcp_anchor"
path_body = "/home/dataproc/postings_gcp_body"
path_title = "/home/dataproc/postings_gcp_title"

In [9]:
# Read indexes
body_index = InvertedIndex.read_index('.',"body_index")
anchor_index = InvertedIndex.read_index('.',"anchor_index")
title_index = InvertedIndex.read_index('.',"title_index")

In [10]:
body_index.bin_path = path_body
anchor_index.bin_path = path_anchor
title_index.bin_path = path_title

In [11]:
with open('doc_id_title_dict.pickle','rb') as f:
    doc_title_dict = pickle.load(f)
    
with open('doc_len_dict.pickle','rb') as f:
    doc_len_dict = pickle.load(f)
    
with open('page_views_dict.pkl','rb') as f:
    page_views_dict = pickle.load(f)

with open('page_rank_dict.pickle','rb') as f:
    page_rank_dict = pickle.load(f)

with open('doc_norm_dict.pickle','rb') as f:
    doc_norm_dict = pickle.load(f)

In [12]:
from contextlib import closing
def read_posting_list(inverted, w, index_type=""):
    with closing(MultiFileReader()) as reader:
        locs = inverted.posting_locs[w]
        b = reader.read(locs, inverted.df[w] * TUPLE_SIZE,index_type)
        posting_list = []
        for i in range(inverted.df[w]):
            doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
            tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
            posting_list.append((doc_id, tf))
        return posting_list

In [13]:
# Reading posting list example:
# lst = read_posting_list(body_index,"texas", path_body)
# print(len(lst))

In [14]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

def tokenize(text):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = set(tok for tok in tokens if tok not in all_stopwords)
    return tokens



In [15]:
def search_title():
    ''' Returns ALL (not just top 100) search results that contain A QUERY WORD 
        IN THE TITLE of articles, ordered in descending order of the NUMBER OF 
        QUERY WORDS that appear in the title. For example, a document with a 
        title that matches two of the query words will be ranked before a 
        document with a title that matches only one query term. 

        Test this by navigating to the a URL like:
         http://YOUR_SERVER_DOMAIN/search_title?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of ALL (not just top 100) search results, ordered from best to 
        worst where each element is a tuple (wiki_id, title).
    '''

    # query=["hello world"] - > [(wiki_id, title)] sorted decreasing
    # in a way that the title that contains most words of the query

    res = []
#     query = request.args.get('hello world', '')
    query = "LinkedIn"
#     query = "hello"
    if len(query) == 0:
        return jsonify(res)
    # BEGIN SOLUTION

    counter = collections.Counter()
    for token in tokenize(query):
        for docId_tf in title_index.read_posting_list(token):
            doc_id = docId_tf[0]
            title = doc_title_dict[doc_id]
            counter[(doc_id, title)] += 1

    
#     res = [map(lambda tup:tup[0], counter.most_common())] 
    res = list(map(lambda tup:tup[0], counter.most_common()))
    # END SOLUTION
    return res

In [54]:
print(search_title())

[(970755, 'LinkedIn'), (27769500, 'LinkedIn Pulse'), (36070366, '2012 LinkedIn hack'), (41726116, 'LinkedIn Learning'), (50191962, 'Timeline of LinkedIn'), (57147095, 'LinkedIn Top Companies'), (62976368, 'HiQ Labs v. LinkedIn')]


In [15]:
def search_anchor():
    # query=["hello world"] - > [(wiki_id, title)] sorted decreasing
    # in a way that the title that contains most words of the query

    res = []
#     query = request.args.get('hello world', '')
    query = "LinkedIn"
#     query = "hello"
    if len(query) == 0:
        return jsonify(res)
    # BEGIN SOLUTION

    counter_anchor = collections.Counter()
    for token in tokenize(query):
        for docId_tf in anchor_index.read_posting_list(token):
            doc_id = docId_tf[0]
            title = doc_title_dict[doc_id]
            counter_anchor[(doc_id, title)] += 1

    
#     res = [map(lambda tup:tup[0], counter.most_common())] 
    res = list(map(lambda tup:tup[0], counter_anchor.most_common()))
    # END SOLUTION
    return res

In [31]:
# print(search_anchor())

In [16]:
def get_pageview():
    ''' Returns the number of page views that each of the provide wiki articles
        had in August 2021.

        Test this by issuing a POST request to a URL like:
          http://YOUR_SERVER_DOMAIN/get_pageview
        with a json payload of the list of article ids. In python do:
          import requests
          requests.post('http://YOUR_SERVER_DOMAIN/get_pageview', json=[1,5,8])
        As before YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of ints:
          list of page view numbers from August 2021 that correrspond to the 
          provided list article IDs.
    '''

    res = []
#     wiki_ids = request.get_json()
    wiki_ids = [848, 21105323, 23536538, 23797577]
    if len(wiki_ids) == 0:
        return res
    # BEGIN SOLUTION
    
    res = [page_views_dict.get(doc_id, -1) for doc_id in wiki_ids]

    # END SOLUTION
    return res

In [62]:
print(get_pageview())

[84370, 1042, 30483, 38197]


In [34]:
def get_pagerank():
    ''' Returns PageRank values for a list of provided wiki article IDs. 

        Test this by issuing a POST request to a URL like:
          http://YOUR_SERVER_DOMAIN/get_pagerank
        with a json payload of the list of article ids. In python do:
          import requests
          requests.post('http://YOUR_SERVER_DOMAIN/get_pagerank', json=[1,5,8])
        As before YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of floats:
          list of PageRank scores that correrspond to the provided article IDs.
    '''
    res = []
#     wiki_ids = request.get_json()
    wiki_ids = [848, 3434750, 32927]
    if len(wiki_ids) == 0:
        return jsonify(res)
    # BEGIN SOLUTION

    res = [page_rank_dict.get(doc_id, -1) for doc_id in wiki_ids]
    # END SOLUTION
#     return jsonify(res)
    return res

In [35]:
print(get_pagerank())

[50.7020900020747, 9913.728782160773, 5282.081575765278]


In [17]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

N = 6348910

def normalize(tokens):
    counter = Counter(tokens)
    norm = 0
    for value in counter.values():
        norm += value*value
    if norm == 0 :
        return 0
    return 1/(math.sqrt(norm))


def tf_idf_calc (token, doc_id, tf):
    tf =  tf/doc_len_dict[doc_id]
#     idf = math.log(N/app.body_index.df[token], 2)
    idf = math.log(N/body_index.df[token], 2)
    return tf*idf

def tokenize(text):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = [tok for tok in tokens if tok not in all_stopwords]
    return tokens

In [27]:
def search_body():
    ''' Returns up to a 100 search results for the query using TFIDF AND COSINE
        SIMILARITY OF THE BODY OF ARTICLES ONLY. DO NOT use stemming. DO USE the 
        staff-provided tokenizer from Assignment 3 (GCP part) to do the 
        tokenization and remove stopwords. 

        To issue a query navigate to a URL like:
         http://YOUR_SERVER_DOMAIN/search_body?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of up to 100 search results, ordered from best to worst where each 
        element is a tuple (wiki_id, title).
    '''
    res = []
#     query = request.args.get('query', '')
    query = 'apple computer'
    if len(query) == 0:
        return jsonify(res)
    # BEGIN SOLUTION

    similarity_dict = {}
    tokens = tokenize(query)
    for token in tokens:
        for doc_tf in body_index.read_posting_list(token):
            if doc_tf[0] not in similarity_dict:
                similarity_dict[doc_tf[0]] = 0
            similarity_dict[doc_tf[0]] += tf_idf_calc(token, doc_tf[0], doc_tf[1])

    for doc in similarity_dict.keys():
        similarity_dict[doc] = similarity_dict[doc]*normalize(tokens)*doc_norm_dict[doc]
        
    top100 = list(sorted(similarity_dict.items(), key=lambda item: item[1], reverse=True)[:100])
    
    for pair in top100:
        res.append((pair[0], doc_title_dict[pair[0]]))

    # END SOLUTION
#     return jsonify(res)
#     return res
    return top100




In [18]:
# print(search_body())

In [19]:
!git clone https://github.com/facebookresearch/fastText.git

Cloning into 'fastText'...
remote: Enumerating objects: 3930, done.
remote: Counting objects: 100% (944/944), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 3930 (delta 854), reused 804 (delta 804), pack-reused 2986
Receiving objects: 100% (3930/3930), 8.24 MiB | 36.07 MiB/s, done.
Resolving deltas: 100% (2505/2505), done.


In [23]:
!cd ..

In [24]:
!pip install fasttext

In [27]:
# import fasttext


model_name = "wiki.en"
# fasttext.download_model(model_name, if_exists='ignore')

fasttext.download_model(model_name, if_exists='ignore',path='/home/dataproc/fasttext_model')



ModuleNotFoundError: No module named 'gensim'

In [ ]:
# Wikipedia 2017 trained Fasttext model 
model_url = 'https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.vec'
response = requests.get(model_url)

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.vec

--2023-01-10 14:53:37--  https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.vec
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6597238061 (6.1G) [binary/octet-stream]
Saving to: ‘wiki.en.vec’

wiki.en.vec         100%[===================>]   6.14G  53.0MB/s    in 2m 10s  

2023-01-10 14:55:48 (48.4 MB/s) - ‘wiki.en.vec’ saved [6597238061/6597238061]



In [ ]:
# # Save the model to a binary file
# for chunk in response.iter_content(chunk_size=1024):
#     if chunk: # filter out keep-alive new chunks
#         f.write(chunk)

# with open("fasttext_model.bin", "wb") as f:
#     f.write(response.content)
    



In [36]:
# Load the model from the binary file
# model = fasttext.load_model("wiki.en.vec")
# model = fasttext.load_model('wiki.en.vec')
model = fasttext.load_model('wiki.en.vec')

TypeError: load_model() got an unexpected keyword argument 'format'

In [32]:

# Use the model to get the vector representation of a word
vector = model.get_word_vector('hello')

# Use the model to get the most similar words to a given word
similar_words = model.get_nearest_neighbors('hello')
print(similar_words)

ValueError: wiki.en.vec has wrong file format!